In [1]:
!pip install -U openai-whisper
!pip install llama_hub
!pip install transformers
!pip install huggingface-cli
!pip install torch
!pip install accelerate
!pip install datasets
!pip install pydub
!pip install bertopic

You should consider upgrading via the '/Users/adityakuppili/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


You should consider upgrading via the '/Users/adityakuppili/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/adityakuppili/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
ERROR: Could not find a version that satisfies the requirement huggingface-cli (from versions: none)
ERROR: No matching distribution found for huggingface-cli
You should consider upgrading via the '/Users/adityakuppili/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/adityakuppili/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/adityakuppili/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


You should consider upgrading via the '/Users/adityakuppili/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/adityakuppili/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


You should consider upgrading via the '/Users/adityakuppili/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


# Topic extraction

I will be using a library called BERTopic that identifies topics being discussed in a text. The output is a list of various topics with the probability of that topic being mentioned in the text.

In [2]:
from bertopic import BERTopic
topic_model = BERTopic.load("davanstrien/chat_topics")

In [3]:
# gets the topics of provided text
def get_topics(text):
    topic, prob = topic_model.transform(text)
    return topic_model.get_topic(topic[0])

In [4]:
get_topics("checkmate is when you win at chess.")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[['chess', 0.531777560710907],
 ['chessboard', 0.503699779510498],
 ['practice', 0.35574236512184143],
 ['strategy', 0.34541815519332886],
 ['learn', 0.34507590532302856],
 ['pawn', 0.30584800243377686],
 ['board', 0.3043401837348938],
 ['pawns', 0.28188765048980713],
 ['play', 0.2593632936477661],
 ['decks', 0.24280010163784027]]

# Transcription

The models I will be evaluating are from HuggingFace. I will be inputing the audio file to the HuggingFace models via pipeline() from transformers. The models I will be evaluating are 
- openai/whisper-large-v2
- facebook/mms-1b-all
- openai/whisper-small.en
- facebook/mms-1b-fl102

In [7]:
import pydub
import time
from transformers import pipeline

# returns array of transcriptions for each chunk
def get_transcriptions(model, chunks):
    start_time = time.time()
    transform_pipe = pipeline("automatic-speech-recognition", model=model)
    transcriptions = []
    for i in range(len(chunks)):
        transform_pipe_res = transform_pipe("./chunk_{}.mp3".format(i))
        transcriptions.append((transform_pipe_res['text'], get_topics(transform_pipe_res['text'])))
    end_time = time.time()
    return transcriptions, end_time - start_time

# splits file into 15 second chunks
def getChunks(file):
    chunks = []
    audio = pydub.AudioSegment.from_mp3(file)
    duration = audio.duration_seconds
    num_chunks = int(duration / 15)
    # Create a list of chunks
    chunks = []
    for i in range(num_chunks):
        start = i * 15000
        end = min((i + 1) * 15000, duration * 1000)
        chunks.append(audio[start:end])
    for i, chunk in enumerate(chunks):
        chunk.export("chunk_{}.mp3".format(i), format="mp3")
    return chunks
    

    

I am using the Chetan_BPO Ops Consultant_Recruiter Manjiri.mp3 file for the transcriptions in this example.

In [8]:
chunks = getChunks('./Chetan_BPO Ops Consultant_Recruiter Manjiri.mp3')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [9]:
openai_transcriptions, openai_time = get_transcriptions("openai/whisper-large-v2", chunks)
for transcription in openai_transcriptions:
    print(transcription[0])
    print(f'Topics: {transcription[1]}')
    print()
print(openai_time)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

 Hello, I am talking to Shripal. Shripal, I am Manjiri. I am calling from E-Zest. I got your application for the BPO position with us couple of days ago. So good time to talk.
Topics: [['letter', 0.4082227051258087], ['sincerely', 0.31927675008773804], ['regards', 0.28891339898109436], ['email', 0.2867659330368042], ['dear', 0.2751607298851013], ['polite', 0.26389122009277344], ['colleague', 0.2621317207813263], ['apologize', 0.255462646484375], ['hi', 0.2506524920463562], ['meeting', 0.21596617996692657]]

 right now? Yeah ma'am. Right Shipal, so like I said I am calling from EZest, we are located in Pondek, Injiawadi, Phase 1, Cubix IT Park and this is a work from your location so we would like to work for you. Yeah. Okay, great so we are
Topics: [['letter', 0.4082227051258087], ['sincerely', 0.31927675008773804], ['regards', 0.28891339898109436], ['email', 0.2867659330368042], ['dear', 0.2751607298851013], ['polite', 0.26389122009277344], ['colleague', 0.2621317207813263], ['apologi

In [10]:
mms_transcriptions, mms_time = get_transcriptions("facebook/mms-1b-all", chunks)
for transcription in mms_transcriptions:
    print(transcription[0])
    print(f'Topics: {transcription[1]}')
    print()
print(mms_time)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

igshrepal an mangery iam calling from eas est i gott ar aplication for the bp o position with us couple of das ago so god taime to tak
Topics: [['letter', 0.4082227051258087], ['sincerely', 0.31927675008773804], ['regards', 0.28891339898109436], ['email', 0.2867659330368042], ['dear', 0.2751607298851013], ['polite', 0.26389122009277344], ['colleague', 0.2621317207813263], ['apologize', 0.255462646484375], ['hi', 0.2506524920463562], ['meeting', 0.21596617996692657]]

solike i said i'm calling from easest we are located in puna ingabarry pased one cubic sity park and this is a work for igabary location soto work for yogra
Topics: [['letter', 0.4082227051258087], ['sincerely', 0.31927675008773804], ['regards', 0.28891339898109436], ['email', 0.2867659330368042], ['dear', 0.2751607298851013], ['polite', 0.26389122009277344], ['colleague', 0.2621317207813263], ['apologize', 0.255462646484375], ['hi', 0.2506524920463562], ['meeting', 0.21596617996692657]]

we are actually a tecnical survise

In [11]:
small_whisper, small_whisper_time = get_transcriptions("openai/whisper-small.en", chunks)
for transcription in small_whisper:
    print(transcription[0])
    print(f'Topics: {transcription[1]}')
    print()
print(small_whisper_time)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

 Hello, I'm talking to Shripal. Shripal, I'm coming from easiest. I got an application for the BPO position with us a couple of days ago. So, time to talk.
Topics: [['letter', 0.4082227051258087], ['sincerely', 0.31927675008773804], ['regards', 0.28891339898109436], ['email', 0.2867659330368042], ['dear', 0.2751607298851013], ['polite', 0.26389122009277344], ['colleague', 0.2621317207813263], ['apologize', 0.255462646484375], ['hi', 0.2506524920463562], ['meeting', 0.21596617996692657]]

 right now? No ma'am. Alright, Chippa. So like I said I'm calling from easiest we are located in Pune, Hindiwadi Phase 1, cubic sitey park and this is a walk from Hindiwadi location so would that work for you? Yeah. Okay great so we are
Topics: [['letter', 0.4082227051258087], ['sincerely', 0.31927675008773804], ['regards', 0.28891339898109436], ['email', 0.2867659330368042], ['dear', 0.2751607298851013], ['polite', 0.26389122009277344], ['colleague', 0.2621317207813263], ['apologize', 0.25546264648437

In [12]:
mms_finetuned, mms_finetuned_time = get_transcriptions("facebook/mms-1b-fl102", chunks)
for transcription in mms_finetuned:
    print(transcription[0])
    print(f'Topics: {transcription[1]}')
    print()
print(mms_finetuned_time)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

tpl  mnry m ci f ezst it a licsn f  bpo positn kits l  ec s0  tim t t
Topics: [['ascii', 0.6627934575080872], ['glyphs', 0.5676862001419067], ['hiragana', 0.4461706280708313], ['art', 0.44102251529693604], ['font', 0.4020988345146179], ['kanji', 0.3987252712249756], ['text', 0.3903907537460327], ['characters', 0.35387447476387024], ['graphics', 0.34570205211639404], ['code', 0.32968831062316895]]

ix d m c fm esst vu od n [uny enjdy fas 1 qubic sity pk n diis  wk f dy otn
Topics: [['ascii', 0.6627934575080872], ['glyphs', 0.5676862001419067], ['hiragana', 0.4461706280708313], ['art', 0.44102251529693604], ['font', 0.4020988345146179], ['kanji', 0.3987252712249756], ['text', 0.3903907537460327], ['characters', 0.35387447476387024], ['graphics', 0.34570205211639404], ['code', 0.32968831062316895]]

tecnicl ​vic dilivry cyeny by  l  bpo  trv0 dom f intrxn0 trvl domn i skl s b v pr itrns p y  3ig  h 9
Topics: [['ascii', 0.6627934575080872], ['glyphs', 0.5676862001419067], ['hiragana', 0.44

In [13]:
import pandas as pd

In [17]:
df = pd.DataFrame({'Model': ['Open AI Whisper Large', 'Facebook MMS', 'Open AI Whisper Small', 'Facebook MMS Finetuned'], 
                   'Time to Transcript in Seconds': [openai_time, mms_time, small_whisper_time, mms_finetuned_time],
                   'Relative Accuracy': ['high', 'low', 'high', 'low']})
df

,Model,Time to Transcript in Seconds,Relative Accuracy
0,Open AI Whisper Large,413.724876,high
1,Facebook MMS,209.307142,low
2,Open AI Whisper Small,78.567409,high
3,Facebook MMS Finetuned,212.309015,low


Based on the accuracy and time of execution of the model, the Whisper Small model would be the best model to use because of the high accuracy and fast execution.